In [8]:
import requests

In [29]:
SCRAPE_SEM = "FA23"

# pull all class subject areas
res = requests.get(f'https://classes.cornell.edu/api/2.0/config/subjects.json?roster={SCRAPE_SEM}').json()
res = res["data"]["subjects"]
res

[{'value': 'AAP',
  'descr': 'Architecture, Art, and Plannin',
  'descrformal': 'Architecture, Art, and Planning'},
 {'value': 'AAS',
  'descr': 'Asian American Studies',
  'descrformal': 'Asian American Studies'},
 {'value': 'AEM',
  'descr': 'Applied Economics & Management',
  'descrformal': 'Applied Economics & Management'},
 {'value': 'AEP',
  'descr': 'Applied & Engineering Physics',
  'descrformal': 'Applied & Engineering Physics'},
 {'value': 'AIIS',
  'descr': 'American Indian & Indigenous',
  'descrformal': 'American Indian and Indigenous Studies'},
 {'value': 'AIRS',
  'descr': 'Air Force Science',
  'descrformal': 'Aerospace Studies'},
 {'value': 'AKKAD', 'descr': 'Akkadian', 'descrformal': 'Akkadian'},
 {'value': 'ALS',
  'descr': 'Agriculture & Life Sciences',
  'descrformal': 'Agriculture & Life Sciences'},
 {'value': 'AMST',
  'descr': 'American Studies',
  'descrformal': 'American Studies'},
 {'value': 'ANSC', 'descr': 'Animal Science', 'descrformal': 'Animal Science'},

In [33]:
from collections import defaultdict
import json
import os

class_data_path = f'json/{SCRAPE_SEM}/scraped_classes.json'

subj_classes = None
if not os.path.exists(class_data_path):
    subj_codes = list(map(lambda s_dict : s_dict["value"], res))
    subj_classes = defaultdict(list)
    for subj in subj_codes:
        print(f'Scraping {subj}...')
        res = requests.get(f'https://classes.cornell.edu/api/2.0/search/classes.json?roster={SCRAPE_SEM}&subject={subj}').json()
        res = res['data']['classes']
        for cls in res:
            if cls["description"]:
                subj_classes[subj].append((subj, cls["catalogNbr"], cls["titleShort"], cls["description"], cls["titleLong"]))
        print(f'Scraped {subj}')
        
        json.dump(subj_classes, open(class_data_path, 'w'))
else:
    subj_classes = json.load(open(class_data_path, 'r'))

In [34]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from tqdm import tqdm

def clean(text):
    text = re.sub(r'\|\|\|', r' ', text)
    text = text.replace('„','')
    text = text.replace('“','')
    text = text.replace('"','')
    text = text.replace('\ax0','')
    text = text.replace('\\', '')
    text = text.replace('\'','')
    text = text.replace('-',' ')
    text = text.replace('&', 'and')
    return text

tagged_descs = []
cmp_classes = []
for i, cls_arr in enumerate(tqdm(subj_classes.values())):
    for dep, code, title, desc, title_full in cls_arr:
        if desc and len(desc) > 100:
            cmp_classes.append((dep, code, title, title_full, clean(desc)))

100%|███████████████████████████████████████| 181/181 [00:00<00:00, 9510.30it/s]


In [35]:
json.dump(cmp_classes, open(f'json/{SCRAPE_SEM}/parsed_courses.json', 'w'))